## Configurações


In [1]:
from osgeo import gdal, osr
import logging
import os
import numpy
import json
from datetime import date, datetime, timedelta
from sklearn.metrics import confusion_matrix
import glob
import pandas
import sys

ModuleNotFoundError: No module named '_gdal'

In [10]:
FORMAT = '[%(asctime)s][%(levelname)s]  %(message)s'
logging.basicConfig(format=FORMAT, level=logging.DEBUG)

In [11]:
path_map1 = "inventario_pampa_byte.tif"
path_map2 = "mapbiomas_pampa_byte.tif"

In [12]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.bool_):
            return bool(obj)
        if isinstance(obj, (numpy.floating, numpy.complexfloating)):
            return float(obj)
        if isinstance(obj, numpy.integer):
            return int(obj)
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        if isinstance(obj, numpy.string_):
            return str(obj)
        if isinstance(obj, (datetime, date)):
            return obj.isoformat()
        if isinstance(obj, timedelta):
            return str(obj)
        return super(NpEncoder, self).default(obj)

### Opcional


In [13]:
mapbiomas_col7_legenda = {
    1: "Floresta",
    3: "Formação Florestal",
    4: "Formação Savânica",
    5: "Mangue",
    49: "Restinga Arborizada",
    10: "Formação Natural não Florestal",
    11: "Campo Alagado e Área Pantanosa",
    12: "Formação Campestre",
    32: "Apicum",
    29: "Afloramento Rochoso",
    50: "Restinga Herbácea",
    13: "Outras Formações não Florestais",
    14: "Agropecuária",
    15: "Pastagem",
    18: "Agricultura",
    19: "Lavoura Temporária",
    39: "Soja",
    20: "Cana",
    40: "Arroz (beta)",
    62: "Algodão (beta)",
    41: "Outras Lavouras Temporárias",
    36: "Lavoura Perene",
    46: "Café",
    47: "Citrus",
    48: "Outras Lavouras Perenes",
    9: "Silvicultura",
    21: "Mosaico de Usos",
    22: "Área não Vegetada",
    23: "Praia, Duna e Areal",
    24: "Área Urbanizada",
    30: "Mineração",
    25: "Outras Áreas não Vegetadas",
    26: "Corpo D'água",
    33: "Rio, Lago e Oceano",
    31: "Aquicultura",
    27: "Não Observado"
}

quarto_inventario_legenda = {
    1: "Floresta manejada (FM)",
    2: "Floresta não manejada (FNM)",
    3: "Floresta secundária (FSec)",
    4: "Corte seletivo (CS)",
    5: "Reflorestamento (Ref)",
    6: "Campo manejado (GM)",
    7: "Campo não manejado (GNM)",
    8: "Campo secundário (GSec)",
    9: "Pastagem (Ap)",
    10: "Pastagem degradada (APD)",
    11: "Outras formações lenhosas manejadas (OFLM)",
    12: "Outras formações lenhosas não manejadas (OFLNM)",
    13: "Outras formações lenhosas secundárias (OFLSec)",
    14: "Agricultura anual (AC)",
    15: "Agricultura perene (PER)",
    16: "Agricultura semiperene (CANA)",
    17: "Água (A)",
    18: "Reservatório (Res)",
    19: "Assentamento (S)",
    20: "Dunas manejadas (DnM)",
    21: "Dunas não manejadas (DnNM)",
    22: "Afloramento rochoso manejado (ArM)",
    23: "Afloramento rochoso não manejado (ArNM)",
    24: "Mineração (Min)",
    25: "Solo exposto (SE)",
    26: "Áreas não observadas (NO)"
}

quarto_inventario_legenda = {
    1: "Managed Forest (FM)",
    2: "Unmanaged Forest (UF)",
    3: "Secondary Forest (SF)",
    4: "Selective Logging (SL)",
    5: "Reforestation (Ref)",
    6: "Managed Field (GM)",
    7: "Unmanaged Field (UF)",
    8: "Secondary Field (SF)",
    9: "Pasture (P)",
    10: "Degraded Pasture (DP)",
    11: "Other Managed Woody Formations (OMWF)",
    12: "Other Unmanaged Woody Formations (OUWF)",
    13: "Other Secondary Woody Formations (OSWF)",
    14: "Annual Agriculture (AA)",
    15: "Perennial Agriculture (PA)",
    16: "Semi-perennial Agriculture (SPA)",
    17: "Water (W)",
    18: "Reservoir (R)",
    19: "Settlement (S)",
    20: "Managed Dunes (MD)",
    21: "Unmanaged Dunes (UD)",
    22: "Managed Rock Outcrop (MRO)",
    23: "Unmanaged Rock Outcrop (URO)",
    24: "Mining (Min)",
    25: "Exposed Soil (ES)",
    26: "Unobserved Areas (UA)"
}

mapbiomas_col7_legenda = {
    1: "Forest",
    3: "Forest Formation",
    4: "Savanna Formation",
    5: "Mangrove",
    49: "Wooded Restinga",
    10: "Natural Non-Forest Formation",
    11: "Wetland",
    12: "Grassland",
    32: "Apicum",
    29: "Rock Outcrop",
    50: "Herbaceous Restinga",
    13: "Other non Forest Formations",
    14: "Agriculture and Livestock",
    15: "Pasture",
    18: "Agriculture",
    19: "Temporary Crop",
    39: "Soybean",
    20: "Sugarcane",
    40: "Rice (beta)",
    62: "Cotton (beta)",
    41: "Other Temporary Crops",
    36: "Perennial Crop",
    46: "Coffee",
    47: "Citrus",
    48: "Other Perennial Crops",
    9: "Silviculture",
    21: "Mosaic of Uses",
    22: "Non-Vegetated Area",
    23: "Beach, Dune, and Sand Spot",
    24: "Urbanized Area",
    30: "Mining",
    25: "Other non Vegetated Areas",
    26: "Water Body",
    33: "River, Lake, and Ocean",
    31: "Aquaculture",
    27: "Not Observed"
}

## Checar parâmetros


In [32]:
def check_maps_param(path_map1: str, path_map2: str):

    # Carrega os arquivos
    dataset1 = gdal.Open(path_map1)
    dataset2 = gdal.Open(path_map2)

    # verifica se as dimensões dos arquivos são iguais
    if dataset1.RasterXSize != dataset2.RasterXSize or dataset1.RasterYSize != dataset2.RasterYSize:
        logging.error("As dimensões dos arquivos são diferentes.")
        return False

    # verifica se as projeções dos arquivos são iguais
    if osr.SpatialReference(dataset1.GetProjection()).GetAttrValue('AUTHORITY', 1) != osr.SpatialReference(dataset2.GetProjection()).GetAttrValue('AUTHORITY', 1):
        logging.error("As projeções dos arquivos são diferentes.")
        return False

    # verifica se as resoluções dos arquivos são iguais
    if round(dataset1.GetGeoTransform()[1], 4) != round(dataset1.GetGeoTransform()[1], 4):
        logging.error("As resoluções dos arquivos são diferentes.")
        return False

    # se tudo estiver correto, exibe uma mensagem de confirmação
    logging.info("Os arquivos têm as mesmas dimensões, projeção e resolução.")

    return True

In [33]:
check_maps_param(path_map1=path_map1, path_map2=path_map2)

[2023-11-25 17:58:38,159][INFO]  Os arquivos têm as mesmas dimensões, projeção e resolução.


True

## Dividir os mapas em blocos de 100MB e salva os arquivos `.csv` da tabulação cruzada


In [15]:
def get_map_name(path_map: str):

    map_name = path_map.split(sep="/")[-1].split(sep=".")[0]
    return map_name

ter uma opção nessa função de salvar o arquivo como um csv


In [46]:
def get_cross_tables(path_map1: str, path_map2: str):

    # Maximum block size in bytes (100 MB)
    max_block_size = 100 * 1024 * 1024

    # Function to calculate the size in bytes of a raster file
    def get_raster_size(path_map1: str):
        file_size = os.path.getsize(path_map1)
        logging.debug(file_size)
        return file_size

    # Function to calculate the number of blocks based on the desired maximum size
    def calculate_num_blocks(path_map1: str, max_block_size: int):
        raster_size = get_raster_size(path_map1=path_map1)
        num_blocks = int(numpy.ceil(raster_size / max_block_size))
        return num_blocks

    # Read files
    dataset1 = gdal.Open(path_map1)
    dataset2 = gdal.Open(path_map2)

    # Get the file dimensions
    cols = dataset1.RasterXSize
    rows = dataset1.RasterYSize

    logging.debug(
        f"Number of rows: {cols}, number of columns: {cols}, map size: {cols * rows} pixels.")

    # Calculate the number of blocks based on the desired maximum size
    num_blocks = calculate_num_blocks(
        path_map1=path_map1, max_block_size=max_block_size)

    num_blocks = int(numpy.ceil(num_blocks/2))

    logging.debug(f"Number of 100mb blocks: {num_blocks*num_blocks}.")

    # Calculate the block width and height
    step_col = int(cols / num_blocks)
    step_row = int(rows / num_blocks)

    logging.debug(f"Block height: {step_row}, block width: {step_col}.")

    # Create an empty dictonary that saves the class order of each block
    class_list = {}

    # Create an empty folder where the cross tables will be save
    path = "./temp_cross_tables"

    # check whether directory already exists
    if not os.path.exists(path):
        os.mkdir(path)

    # create a block count
    index = 0

    for i in range(0, rows, step_row):
        for j in range(0, cols, step_col):

            # Calculate the block size for this part
            width = min(step_col, cols - j)
            height = min(step_row, rows - i)

            # Read the blocks with the determined sizes
            block1 = dataset1.ReadAsArray(j, i, xsize=width, ysize=height)
            block1 = block1.reshape(-1)
            logging.debug(f"Bloco 1: {sys.getsizeof(block1)}.")

            block2 = dataset2.ReadAsArray(j, i, xsize=width, ysize=height)
            block2 = block2.reshape(-1)
            logging.debug(f"Bloco 2: {sys.getsizeof(block2)}.")

            # Calculate the unique pixel values for each map
            classes1 = numpy.unique(block1)
            classes2 = numpy.unique(block2)

            # Concatenate the values and generate the list of classes
            classes = numpy.unique(numpy.concatenate((classes1, classes2)))

            class_list[f"{i}_{j}"] = list(classes)

            # Calculate the confusion matrix
            block_confusion_matrix = confusion_matrix(
                block1, block2, labels=classes)

            # Get the main name of the map files
            name_map1 = get_map_name(path_map=path_map1)
            name_map2 = get_map_name(path_map=path_map2)

            # Save the confusion matrix to a CSV file
            numpy.savetxt(
                f'./temp_cross_tables/{name_map1}_{name_map2}_{i}_{j}.csv', block_confusion_matrix, delimiter=',')

            index += 1
            logging.debug(
                f"Process in {round(index/(num_blocks * num_blocks)  * 100, 2)}%.")

    with open('./temp_cross_tables/class_list.txt', 'w') as fout:
        fout.write(json.dumps(class_list, cls=NpEncoder, indent=2))

In [47]:
get_cross_tables(path_map1=path_map1, path_map2=path_map2)

[2023-11-25 18:18:49,199][DEBUG]  Number of rows: 29578, number of columns: 29578, map size: 622853524 pixels.
[2023-11-25 18:18:49,200][DEBUG]  622980255
[2023-11-25 18:18:49,201][DEBUG]  Number of 100mb blocks: 9.
[2023-11-25 18:18:49,203][DEBUG]  Block height: 7019, block width: 9859.
[2023-11-25 18:18:49,299][DEBUG]  Bloco 1: 112.
[2023-11-25 18:18:49,443][DEBUG]  Bloco 2: 112.
[2023-11-25 18:19:19,324][DEBUG]  Process in 11.11%.
[2023-11-25 18:19:19,339][DEBUG]  Bloco 1: 112.
[2023-11-25 18:19:19,350][DEBUG]  Bloco 2: 112.
[2023-11-25 18:19:50,759][DEBUG]  Process in 22.22%.
[2023-11-25 18:19:50,778][DEBUG]  Bloco 1: 112.
[2023-11-25 18:19:50,798][DEBUG]  Bloco 2: 112.
[2023-11-25 18:20:22,074][DEBUG]  Process in 33.33%.
[2023-11-25 18:20:22,077][DEBUG]  Bloco 1: 112.
[2023-11-25 18:20:22,081][DEBUG]  Bloco 2: 112.
[2023-11-25 18:20:22,090][DEBUG]  Process in 44.44%.
[2023-11-25 18:20:22,316][DEBUG]  Bloco 1: 112.
[2023-11-25 18:20:22,405][DEBUG]  Bloco 2: 112.
[2023-11-25 18:20:5

## Criar a matriz de tabulação cruzada com todos os blocos extraidos


In [7]:
def unify_cross_tables(path_map1: str, path_map2: str, null_value=0.0):

    # aqui quero que o null value possa ser uma lista e precisa verificar
    # se esse valor existe no index ou nas colunas antes de apagar
    # ou vai dar erro

    # read the class list created
    with open('./temp_cross_tables/class_list.txt', 'r') as fout:
        class_list = fout.read()

    # Convert the jason file to a dictonary
    class_list = json.loads(class_list)

    # Select the unique pixels values in all cross tables generated
    class_items = []
    for key, value in class_list.items():
        class_items = numpy.unique(numpy.concatenate((class_items, value)))

    # create an empty cross table with the correspondant pixel values from both maps
    cross_table = pandas.DataFrame(0, index=class_items, columns=class_items)

    # Select all the csv files in the cross table folder
    files = glob.glob("./temp_cross_tables/*.csv")

    # Get the main name of the map files
    name_map1 = get_map_name(path_map=path_map1)
    name_map2 = get_map_name(path_map=path_map2)

    for filename in files:
        classes = filename.split(f"{name_map1}_{name_map2}_")[
            1].split(".csv")[0]
        cross_table_block = pandas.read_csv(
            filename, names=class_list[classes]).set_axis(class_list[classes], axis='index')

        for column in cross_table_block.columns:
            for index in cross_table_block.index:
                cross_table.at[index, column] = cross_table.at[index,
                                                               column] + cross_table_block.at[index, column]

    cross_table = cross_table.loc[(cross_table != 0.0).any(axis=1)]
    cross_table = cross_table.loc[:, (cross_table != 0.0).any(axis=0)]

    cross_table = cross_table.drop(index=null_value, columns=null_value)

    return cross_table

In [25]:
cross_table = unify_cross_tables(path_map1=path_map1, path_map2=path_map2)

In [26]:
cross_table

,3.0,9.0,11.0,12.0,15.0,21.0,23.0,24.0,25.0,29.0,30.0,31.0,33.0,39.0,40.0,41.0,49.0,50.0,255.0
1.0,397121,12846,386079,2489282,95,33389,1641,512,8911,19783,0,0,40628,9739,5092,2914,49711,67215,690
2.0,11816603,814950,2271101,17429557,16422,867829,83339,42649,226362,156022,1109,0,853746,980790,141984,62668,436980,714203,89495
3.0,347343,54639,94466,954335,1784,128557,114,2527,8850,5067,8110,0,45079,111117,22889,11553,19406,11114,2210
5.0,2255892,7128947,30877,1243334,3494,117580,20392,3452,60024,1604,5025,0,18396,114695,4449,8079,106707,87232,26203
6.0,57017,1111,5014,503860,0,5847,0,0,2921,1859,0,0,1091,4078,370,304,0,0,0
7.0,3635903,114050,79615,16525140,44,260810,248,943,51510,44410,1823,0,66599,416506,7252,24140,312,0,2544
8.0,153483,9889,8760,782522,10,31506,171,71,10598,146,42,0,9716,44844,573,3951,13,0,222
9.0,3655986,460566,585268,35928287,114536,2889428,15798,65060,497507,82942,4281,0,325793,3420466,176132,656813,153103,432109,14653
11.0,2651,0,1031,8353,0,3083,0,0,0,0,0,0,342,145,92,129,0,0,0
12.0,17282,0,126,104831,0,555,0,0,70,764,0,0,382,2040,330,37,0,0,90


## Função opcional: mudar o nome das classes a partir de um dicionario dado


In [28]:
def get_class_names(df, class_map_1: dict, class_map_2: dict):

    # Getting pixels
    pixels_map1 = df.index.astype(float)
    pixels_map2 = df.columns

    # Mapping pixel values to class names
    legend_map1 = list(map(class_map_1.get, pixels_map1))
    legend_map2 = list(map(class_map_2.get, pixels_map2))

    # Renaming rows and columns according to class legend
    df.index = legend_map1
    df.columns = legend_map2

    return df

In [29]:
cross_table = get_class_names(
    cross_table, quarto_inventario_legenda, mapbiomas_col7_legenda)

In [30]:
cross_table

,Forest Formation,Silviculture,Wetland,Grassland,Pasture,Mosaic of Uses,"Beach, Dune, and Sand Spot",Urbanized Area,Other non Vegetated Areas,Rock Outcrop,Mining,Aquaculture,"River, Lake, and Ocean",Soybean,Rice (beta),Other Temporary Crops,Wooded Restinga,Herbaceous Restinga,None
Managed Forest (FM),397121,12846,386079,2489282,95,33389,1641,512,8911,19783,0,0,40628,9739,5092,2914,49711,67215,690
Unmanaged Forest (UF),11816603,814950,2271101,17429557,16422,867829,83339,42649,226362,156022,1109,0,853746,980790,141984,62668,436980,714203,89495
Secondary Forest (SF),347343,54639,94466,954335,1784,128557,114,2527,8850,5067,8110,0,45079,111117,22889,11553,19406,11114,2210
Reforestation (Ref),2255892,7128947,30877,1243334,3494,117580,20392,3452,60024,1604,5025,0,18396,114695,4449,8079,106707,87232,26203
Managed Field (GM),57017,1111,5014,503860,0,5847,0,0,2921,1859,0,0,1091,4078,370,304,0,0,0
Unmanaged Field (UF),3635903,114050,79615,16525140,44,260810,248,943,51510,44410,1823,0,66599,416506,7252,24140,312,0,2544
Secondary Field (SF),153483,9889,8760,782522,10,31506,171,71,10598,146,42,0,9716,44844,573,3951,13,0,222
Pasture (P),3655986,460566,585268,35928287,114536,2889428,15798,65060,497507,82942,4281,0,325793,3420466,176132,656813,153103,432109,14653
Other Managed Woody Formations (OMWF),2651,0,1031,8353,0,3083,0,0,0,0,0,0,342,145,92,129,0,0,0
Other Unmanaged Woody Formations (OUWF),17282,0,126,104831,0,555,0,0,70,764,0,0,382,2040,330,37,0,0,90


## Função opcional: converter o valor de pixel para uma área dada


In [ ]:
def convert_pixel(df, conversion_rate: float, decimal_precision=2):

    df = round(df * conversion_rate, decimal_precision)

    return df

## Harmonização


### Parte I: criar a harmonização por linhas


In [37]:
def row_equivalence(cross_table: pandas.DataFrame, name_map1: str = "map1", name_map2: str = "map2"):
    """Essa função determina as esquivalências por linha entre dois mapas de uso e cobertura da terra (UTC).
        Recebe como entrada uma matriz de tablução cruzada entre os dois mapas.
    """

    row_mapping = []
    for index, row in cross_table.iterrows():
        max_row = row.sort_values(ascending=False)
        row_mapping.append(cross_table.loc[:, max_row.index[0]])

    row_mapping = pandas.DataFrame(data=row_mapping)

    linhas = row_mapping.index.tolist()
    colunas = row_mapping.columns.tolist()
    data_tuples = list(zip(colunas, linhas))
    row_mapping = pandas.DataFrame(
        data_tuples, columns=[f"{name_map1}", f"{name_map2}"])

    for i in range(len(row_mapping)):
        row_mapping.loc[i, "size"] = cross_table.loc[(
            row_mapping[f"{name_map1}"][i])].max()

    return row_mapping

In [36]:
row_teste = row_equivalence(
    cross_table=cross_table, name_map1="Quarto_Inventario", name_map2="MapBiomas")

In [34]:
row_teste

,Quarto_Inventario,MapBiomas,size
0,Managed Forest (FM),Grassland,2489282.0
1,Unmanaged Forest (UF),Grassland,17429557.0
2,Secondary Forest (SF),Grassland,954335.0
3,Reforestation (Ref),Silviculture,7128947.0
4,Managed Field (GM),Grassland,503860.0
5,Unmanaged Field (UF),Grassland,16525140.0
6,Secondary Field (SF),Grassland,782522.0
7,Pasture (P),Grassland,35928287.0
8,Other Managed Woody Formations (OMWF),Grassland,8353.0
9,Other Unmanaged Woody Formations (OUWF),Grassland,104831.0


### Parte 2: criar a harmonização por colunas


In [ ]:
def column_equivalence(tabulacao_cruzada):
    """Essa função determina as esquivalências por coluna entre dois mapas de uso e cobertura da terra (UTC).
        Recebe como entrada uma matriz de tablução cruzada entre os dois mapas.
    """

    mapeamento_coluna = []
    for column in range(0, tabulacao_cruzada.shape[1]):
        max_column = tabulacao_cruzada.iloc[:,
                                            column].sort_values(ascending=False)
        mapeamento_coluna.append(tabulacao_cruzada.loc[max_column.index[0], :])

    mapeamento_coluna = pd.DataFrame(data=mapeamento_coluna)

    linhas = mapeamento_coluna.index.tolist()
    colunas = mapeamento_coluna.columns.tolist()
    data_tuples = list(zip(colunas, linhas))
    mapeamento_coluna = pd.DataFrame(
        data_tuples, columns=['MapBiomas', 'Quarto_Inventario'])

    mapeamento_coluna.loc[:, ("Size")] = 0

    for i in range(len(mapeamento_coluna)):
        mapeamento_coluna.loc[i, "Size"] = tabulacao_cruzada.loc[:,
                                                                 (mapeamento_coluna["MapBiomas"][i])].max()

    return mapeamento_coluna